In [15]:
import fiona
import rasterio
import imagesize
import numpy as np
from rasterio.features import rasterize
from fiona.transform import transform as f_transform

base_dir = '/work/OT/ai4geo/DATA/DATASETS/DIGITANIE'
tile_shp = fiona.open(base_dir+"/Montpellier/shapefiles/Montpellier_0.shp")
big_raster = rasterio.open(base_dir+'/Montpellier/MONTPELLIER_20190912_T_TOA_reproj-EPSG:2154.tif')
minx, miny, maxx, maxy = tile_shp.bounds
(minx, maxx), (miny, maxy) = f_transform(tile_shp.crs, dict(big_raster.crs), [minx, maxx], [miny, maxy])
w = rasterio.windows.from_bounds(minx, miny, maxx, maxy, transform=big_raster.transform)
x = np.random.randint(w.col_off, w.col_off+w.width-2000+1)
y = np.random.randint(w.row_off, w.row_off+w.height-2000+1)
chip = rasterio.windows.Window(x, y, 2000, 2000)
minx, miny, maxx, maxy = rasterio.windows.bounds(chip, transform=big_raster.transform)
t = rasterio.transform.from_bounds(minx, miny, maxx, maxy, 2000, 2000)
(minx, maxx), (miny, maxy) = f_transform(dict(big_raster.crs), tile_shp.crs, [minx, maxx], [miny, maxy])

In [21]:
shapes_for_raster = [(s['geometry'], int(s['properties']['num_class'])) for s in tile_shp.filter(bbox=(minx, miny, maxx, maxy))]

In [22]:
print(shapes_for_raster[0])

({'type': 'Polygon', 'coordinates': [[(772757.0688552876, 6277716.534484411), (772762.1978812575, 6277712.631964651), (772763.089885774, 6277713.858470861), (772772.3759126113, 6277705.354425231), (772772.5964619346, 6277705.152448482), (772773.6824394075, 6277704.157921744), (772764.873894807, 6277694.011370368), (772763.3093166724, 6277695.27950212), (772754.2813411736, 6277702.59691384), (772754.9282291353, 6277703.479033788), (772755.5078473837, 6277704.269422309), (772750.3788214138, 6277707.837440374), (772757.0688552876, 6277716.534484411)]]}, 4)


In [24]:
profile = big_raster.profile
profile.update(dtype=rasterio.uint8, count=3, width=2000, height=2000, transform=t, compress='lzw')
with rasterio.Env():
    with rasterio.open('test.tif', 'w', **profile) as dst:
        for i, att in enumerate(['num_class']):
            shapes_for_raster = [(s['geometry'], int(s['properties'][att])) for s in tile_shp.filter(bbox=(minx, miny, maxx, maxy))]
            masks = rasterize(shapes_for_raster, out_shape=(2000, 2000), transform=t)
            dst.write(masks.astype(rasterio.uint8), i+1)